### First, to parse the dataset

In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time

In [2]:
from bs4 import BeautifulSoup,SoupStrainer

In [3]:
f = open('data/reut2-000.sgm', 'r')
data= f.read()
soup = BeautifulSoup(data)

topics = list(soup.findAll('topics'))
text = list(soup.findAll('text'))

/Users/serbanstan/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/serbanstan/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Keep fewer than max entries. Can comment out the next cell after more rigorous testing.

In [4]:
topics = topics[:200]
text = text[:200]

In [5]:
print topics[11].text
print text[11].text

earnacq

OHIO MATTRESS <OMT> MAY HAVE LOWER 1ST QTR NET
    CLEVELAND, Feb 26 - Ohio Mattress Co said its first
quarter, ending February 28, profits may be below the 2.4 mln
dlrs, or 15 cts a share, earned in the first quarter of fiscal
1986.
    The company said any decline would be due to expenses
related to the acquisitions in the middle of the current
quarter of seven licensees of Sealy Inc, as well as 82 pct of
the outstanding capital stock of Sealy.
    Because of these acquisitions, it said, first quarter sales
will be substantially higher than last year's 67.1 mln dlrs.
    Noting that it typically reports first quarter results in
late march, said the report is likely to be issued in early
April this year.
    It said the delay is due to administrative considerations,
including conducting appraisals, in connection with the
acquisitions.
 Reuter



In [6]:
uniqueTopics = {}

for topicList in topics:
    for t in topicList:
        if t.text != '':
            uniqueTopics[t.text] = True

print len(uniqueTopics)
print uniqueTopics

35
{u'copper': True, u'livestock': True, u'sunseed': True, u'trade': True, u'cocoa': True, u'reserves': True, u'soybean': True, u'ship': True, u'cotton': True, u'housing': True, u'rye': True, u'gnp': True, u'sugar': True, u'veg-oil': True, u'oat': True, u'barley': True, u'crude': True, u'coffee': True, u'soy-oil': True, u'wheat': True, u'meal-feed': True, u'corn': True, u'nat-gas': True, u'oilseed': True, u'linseed': True, u'lin-oil': True, u'earn': True, u'cpi': True, u'soy-meal': True, u'sun-oil': True, u'money-supply': True, u'carcass': True, u'acq': True, u'grain': True, u'sorghum': True}


Make these things callable by index

In [7]:
topicsToInd = dict()
indToTopic = dict()

for i in range(len(uniqueTopics)):
    topicsToInd[uniqueTopics.keys()[i]] = i
    indToTopic[i] = uniqueTopics.keys()[i]

Next, keep only the articles that have a non-null list of topics

In [8]:
valid = []

for ind in range(len(topics)):
    if len(topics[ind]) != 0:
        valid.append(ind)
        
print len(valid)
print [t.text for t in topics[valid[1]]]

126
[u'grain', u'wheat', u'corn', u'barley', u'oat', u'sorghum']


In [9]:
topics = [topics[x] for x in valid]
for ind in range(len(topics)):
    topicList = topics[ind]
    toText = []
    
    for t in topicList:
        toText.append(t.text)
        
    topics[ind] = toText[:]

text = [text[x].text for x in valid]

In [10]:
print topics[6]
print text[6]

[u'earn', u'acq']

OHIO MATTRESS <OMT> MAY HAVE LOWER 1ST QTR NET
    CLEVELAND, Feb 26 - Ohio Mattress Co said its first
quarter, ending February 28, profits may be below the 2.4 mln
dlrs, or 15 cts a share, earned in the first quarter of fiscal
1986.
    The company said any decline would be due to expenses
related to the acquisitions in the middle of the current
quarter of seven licensees of Sealy Inc, as well as 82 pct of
the outstanding capital stock of Sealy.
    Because of these acquisitions, it said, first quarter sales
will be substantially higher than last year's 67.1 mln dlrs.
    Noting that it typically reports first quarter results in
late march, said the report is likely to be issued in early
April this year.
    It said the delay is due to administrative considerations,
including conducting appraisals, in connection with the
acquisitions.
 Reuter



In [11]:
print len(topics), len(text), "   <---- these should be equal"

126 126    <---- these should be equal


Next, create a dictionary of type topics -> article indexes with said topics

In [12]:
articles = dict()

for i in range(len(topics)):
    for t in topics[i]:
        if topicsToInd[t] in articles:
            articles[topicsToInd[t]].append(i)
        else:
            articles[topicsToInd[t]] = [i]

print articles[topicsToInd['wheat']]

[1, 2, 10, 59, 112]


### Now, to generate tf-idf scores

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

Set ngram_range for a more refined analysis

In [14]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')

In [15]:
X = vectorizer.fit_transform(text).todense()

In [16]:
for ind in range(len(X)):
    X[ind] = np.array(X[ind].tolist()[0])

### Can we compute the cosine between two different articles? Create a cosine similarity matrix.

In [17]:
cosine_similarity(X[0].reshape(1,-1), X[6].reshape(1,-1))

array([[ 0.03125751]])

In [18]:
n = len(topics)

In [19]:
csim = [[0 for i in range(n)] for i in range(n)]

for i in range(n):
    for j in range(n):
        csim[i][j] = cosine_similarity(X[i].reshape(1,-1), X[j].reshape(1,-1))[0][0]

### Now, apply our algorithm with facility location as the submodular functions

In [20]:
n = len(topics)
m = len(uniqueTopics)
l = 10
k = 5

In [21]:
from novel import novel

startTime = time.time()
myS, myCost, myEvals = novel(n, m, l, k, csim, articles)
print "Algo ran in ", time.time() - startTime
print 'Our solution = ', myCost

# sanity check
# print len(myS)
# for i in range(m):
#     print set(myBestS[i]).issubset(myS), len(myBestS[i]) == k

We obtained objective value  59.0597597683  for set  [2, 59, 18, 125, 1, 39, 44, 122, 98, 61]
Algo ran in  1.56849813461
Our solution =  59.0597597683


In [22]:
from greedymerge import greedymerge

gmS, gmCost, gmEvals = greedymerge(n, m, l, k, csim, articles)

print 'Obtained solution', gmCost

Greedy Merge gives cost =  98.5718681183
Size of S is  47
Obtained solution 98.5718681183


In [23]:
from greedysum import greedysum

gsS, gsCost, gsEvals = greedysum(n, m, l, k, csim, articles)

print 'Greedy Sum gives', gsCost

Greedy Sum gives cost =  59.0555468889
Greedy Sum gives 59.0555468889


In [24]:
# from localsearch import localsearch

# lsS, lsCost, lsEvals = localsearch(n, m, l, k, csim, articles, epsilon = 0.2)

# print 'Local Search gives', lsCost

In [25]:
from replacementGreedy import replacementGreedy

rgS, rgCost, rgEvals = replacementGreedy(n, m, l, k, csim, articles)

print 'Replacement Greedy gives ', rgCost

Replacement Greedy gives  59.0597597683
